# routes

>  Route handlers for the contacts application

In [ ]:
#| default_exp routes

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fasthtml.common import *
from fastcore.test import *
from todo.models import *
from todo.database import *
from todo.components import *

## Helper Functions

In [ ]:
#| export
def is_htmx_request(req):
    "Check if a request is coming from HTMX"
    return req and req.headers.get("HX-Request") == "true"

In [ ]:
default_db?

Type:           ContactDB
String form:    <todo.database.ContactDB object>
File:           ~/github/todo/todo/database.py
Docstring:      Simple in-memory database for contacts
Init docstring: Initialize with optional contacts

## Route Handlers

In [ ]:
#| export
default_db = default_db

def setup_routes(app, rt, db=default_db):
    "Set up all routes for the contacts application"
    
    @rt("/contacts")
    def get_contacts(req=None):
        "Handle requests for the contacts list"
        query = req.query_params.get("q", "") if req else ""
        contacts = db.search(query)
        content = contact_list(contacts, query)
        
        if is_htmx_request(req):
            # If it's an HTMX request, just return the contacts list
            return Div(id="contacts-list")(
                *[contact_card(contact) for contact in contacts]
            ) if contacts else P("No contacts found.")
        
        # Otherwise return the full page
        return page_layout(content, "All Contacts")
    
    @rt("/contacts/favorites")
    def get_favorites(req=None):
        "Handle requests for favorite contacts"
        favorites = [c for c in db.get_all() if c.favorite]
        content = contact_list(favorites)
        
        if is_htmx_request(req):
            return Div(id="contacts-list")(
                *[contact_card(contact) for contact in favorites]
            ) if favorites else P("No favorite contacts.")
        
        return page_layout(content, "Favorite Contacts")
    
    @rt("/contacts/new")
    def new_contact(req=None):
        "Handle requests for the new contact form"
        content = contact_form()
        
        if is_htmx_request(req):
            return content
        
        return page_layout(content, "Add New Contact")
    
    @rt("/contacts")
    def post(req=None, name: str = "", email: str = "", 
             phone: str = "", address: str = "", favorite: bool = False):
        "Handle creating a new contact"
        contact = Contact(0, name, email, phone, address, favorite)
        db.add(contact)
        
        # Redirect to contact list
        return RedirectResponse("/contacts", status_code=303)
    
    @rt("/contacts/{contact_id:int}")
    def get_contact_detail(contact_id: int, req=None):
        "Handle requests for contact details"
        contact = db.get(contact_id)
        if not contact:
            return "Contact not found", 404
        
        content = contact_detail(contact)
        
        if is_htmx_request(req):
            return content
        
        return page_layout(content, f"Contact: {contact.name}")
    
    @rt("/contacts/{contact_id:int}/edit")
    def edit_contact(contact_id: int, req=None):
        "Handle requests for the edit contact form"
        contact = db.get(contact_id)
        if not contact:
            return "Contact not found", 404
        
        content = contact_form(contact, f"/contacts/{contact_id}", "put")
        
        if is_htmx_request(req):
            return content
        
        return page_layout(content, f"Edit: {contact.name}")
    
    @rt("/contacts/{contact_id:int}")
    def put(contact_id: int, name: str = "", email: str = "", 
            phone: str = "", address: str = "", favorite: bool = False):
        "Handle updating an existing contact"
        contact = Contact(contact_id, name, email, phone, address, favorite)
        if not db.update(contact):
            return "Contact not found", 404
        
        # Redirect to contact detail
        return RedirectResponse(f"/contacts/{contact_id}", status_code=303)
    
    @rt("/contacts/{contact_id:int}")
    def delete(contact_id: int):
        "Handle deleting a contact"
        if not db.delete(contact_id):
            return "Contact not found", 404
        
        # Return empty response with success status
        return ""
    
    @rt("/")
    def home():
        "Redirect the home page to contacts list"
        return RedirectResponse("/contacts")

## Testing Route Handlers

We can test our route handlers using a mock request object:

In [ ]:
# export
from fasthtml.common import *

In [ ]:
# # Create a test app with a simple route
# # Use FastHTML's app and router
# app, rt = fast_app()
# test_db = ContactDB(sample_contacts[:])

# # Set up routes
# setup_routes(app, rt, test_db)

# # Create a test client using FastHTML's approach
# from starlette.testclient import TestClient
# client = TestClient(app)

# # Test endpoints
# response = client.get("/contacts")
# test_eq(response.status_code, 200)
# assert "John Doe" in response.text

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()